# instalación de paquetes y verificación

In [1]:
# comprobamos que la versión de python sea inferior a 3.8, ya que para usar tensorflow 1.15 se recomienda usar python
# 3.7 o inferior
import sys
import string
python_version=sys.version.split("(")[0]
if int(sys.version.split(".")[1])>7:
    print("you are using a python version higer than 3.7.x, please install python 3.7.x")
else:
    print("congrats, you are using {} which is compatible with this notebook".format(python_version))


congrats, you are using 3.7.9  which is compatible with this notebook


In [15]:
# si todo sale bien,el test debería correr y los resultados aparecerán debajo. Es normal si se salta alguno de los tests 
# que hay
!python3 models/research/object_detection/builders/model_builder_tf1_test.py


Running tests under Python 3.7.9: /home/bigdata/anaconda3/envs/test/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
/home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/data_structures.py:669: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feat

# creación del dataset

In [16]:
# vamos a dividir nuestras imágenes y anotaciones en splits de train, test y validación.
# los argumentos -vr y -tr indican respectivamente los porcentajes de datos reservados para validación y test
!python3 scripts/train_test_split.py -i images_bb -o images_bb -tr 0.1 -vr 0.15 -f JPG
!tree -d images_bb

succesfully created validation split using:  15.0 % of data
succesfully created test split using:  10.0 % of data
succesfully created train split using:  75.0 % of data
images_bb
├── test
├── train
└── val

3 directories


In [21]:
# convertimos los archivos de anotación en formato json a archivos en formato XML. Recuerda que labelme produce archivos
# json y labe
!python3 scripts/json_to_xml.py -i images_bb/train -o images_bb/train
!python3 scripts/json_to_xml.py -i images_bb/val -o images_bb/val
!python3 scripts/json_to_xml.py -i images_bb/test -o images_bb/test

G0011154.xml
G0021262.xml
G0051599.xml
G0061723.xml
G0051637.xml
G0051597.xml
G0031460.xml
G0061737.xml
G0021319.xml
G0061746.xml
G0051527.xml
G0051642.xml
G0011175.xml
G0031435.xml
G0011145.xml
G0051577.xml
G0021268.xml
G0031483.xml
G0011144.xml
G0061816.xml
G0021333.xml
G0051576.xml
G0031476.xml
G0011193.xml
G0011146.xml
G0061798.xml
G0051680.xml
G0021327.xml
G0061736.xml
G0011227.xml
G0061763.xml
G0031438.xml
G0061740.xml
G0011138.xml
G0051655.xml
G0031439.xml
G0011147.xml
G0041505.xml
G0061717.xml
G0021372.xml
G0061811.xml
G0051572.xml
G0031454.xml
G0011150.xml
G0031452.xml
G0031386.xml
G0061838.xml
G0011159.xml
G0011142.xml
G0031431.xml
G0051631.xml
G0051636.xml
G0031442.xml
G0011210.xml
G0011160.xml
G0061755.xml
G0011148.xml
G0021357.xml
G0061720.xml
G0061696.xml
G0021369.xml
G0061745.xml
G0011139.xml
G0021281.xml
G0041503.xml
G0051645.xml
G0021296.xml
G0011156.xml
G0011141.xml
G0061684.xml
G0031449.xml
G0031414.xml
G0061831.xml
G0011149.xml
G0051538.xml
G0011143.xml
G0051546.xml

In [24]:
# ahora tenemos que generar los tfrecords usando estos dataset en formato coco
!python3 scripts/generate_tfrecord.py -x images_bb/train -l training_bb/label_map.pbtxt -o training_bb/train.record
!python3 scripts/generate_tfrecord.py -x images_bb/test -l training_bb/label_map.pbtxt -o training_bb/test.record
!python3 scripts/generate_tfrecord.py -x images_bb/val -l training_bb/label_map.pbtxt -o training_bb/val.record

Successfully created the TFRecord file: training/train.record
Successfully created the TFRecord file: training/test.record
Successfully created the TFRecord file: training/val.record


# Entrenamiento y exportación del modelo

In [ ]:
# entrenamos el modelo usando estos tfrecords. El modelo resultante se guardará el carpeta training_bb
!python3 models/research/object_detection/model_main.py --alsologtostderr \
--model_dir=training_bb/ \
--pipeline_config_path=training_bb/faster_rcnn_inception_v2_coco.config \
--num_train_steps=1000000

W0910 06:33:40.624029 139802634913600 model_lib.py:771] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 1000000
I0910 06:33:40.624184 139802634913600 config_util.py:552] Maybe overwriting train_steps: 1000000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0910 06:33:40.624230 139802634913600 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0910 06:33:40.624269 139802634913600 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0910 06:33:40.624315 139802634913600 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0910 06:33:40.624367 139802634913600 model_lib.py:787] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, expor

Instructions for updating:
Use `tf.cast` instead.
W0910 06:33:52.026989 139802634913600 deprecation.py:323] From /home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/object_detection/inputs.py:259: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.
I0910 06:33:54.930131 139802634913600 estimator.py:1148] Calling model_fn.
Instructions for updating:
Please use `layer.__call__` method instead.
W0910 06:33:54.950996 139802634913600 deprecation.py:323] From /home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I0910 06:33:55.783972 139802634913600 regularizers.py:99] Scale of 0 

Instructions for updating:
Use standard file utilities to get mtimes.
W0910 06:34:02.520121 139802634913600 deprecation.py:323] From /home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/tensorflow_core/python/training/saver.py:1069: get_checkpoint_mtimes (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
I0910 06:34:03.207053 139802634913600 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0910 06:34:03.383164 139802634913600 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 400000 into training/model.ckpt.
I0910 06:34:07.087545 139802634913600 basic_session_run_hooks.py:606] Saving checkpoints for 400000 into training/model.ckpt.
2020-09-10 06:34:11.270328: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Success

INFO:tensorflow:global_step/sec: 9.2675
I0910 06:38:02.847584 139802634913600 basic_session_run_hooks.py:692] global_step/sec: 9.2675
INFO:tensorflow:loss = 0.030523367, step = 402100 (10.791 sec)
I0910 06:38:02.848835 139802634913600 basic_session_run_hooks.py:260] loss = 0.030523367, step = 402100 (10.791 sec)
INFO:tensorflow:global_step/sec: 9.31549
I0910 06:38:13.582391 139802634913600 basic_session_run_hooks.py:692] global_step/sec: 9.31549
INFO:tensorflow:loss = 0.010457026, step = 402200 (10.735 sec)
I0910 06:38:13.583524 139802634913600 basic_session_run_hooks.py:260] loss = 0.010457026, step = 402200 (10.735 sec)
INFO:tensorflow:global_step/sec: 9.432
I0910 06:38:24.184583 139802634913600 basic_session_run_hooks.py:692] global_step/sec: 9.432
INFO:tensorflow:loss = 0.0046164906, step = 402300 (10.602 sec)
I0910 06:38:24.185793 139802634913600 basic_session_run_hooks.py:260] loss = 0.0046164906, step = 402300 (10.602 sec)
INFO:tensorflow:global_step/sec: 9.28686
I0910 06:38:34.

INFO:tensorflow:global_step/sec: 9.31334
I0910 06:42:42.641354 139802634913600 basic_session_run_hooks.py:692] global_step/sec: 9.31334
INFO:tensorflow:loss = 0.0027870305, step = 404700 (10.737 sec)
I0910 06:42:42.642045 139802634913600 basic_session_run_hooks.py:260] loss = 0.0027870305, step = 404700 (10.737 sec)
INFO:tensorflow:global_step/sec: 9.28732
I0910 06:42:53.408809 139802634913600 basic_session_run_hooks.py:692] global_step/sec: 9.28732
INFO:tensorflow:loss = 0.0039271787, step = 404800 (10.768 sec)
I0910 06:42:53.409948 139802634913600 basic_session_run_hooks.py:260] loss = 0.0039271787, step = 404800 (10.768 sec)
INFO:tensorflow:global_step/sec: 9.21514
I0910 06:43:04.260424 139802634913600 basic_session_run_hooks.py:692] global_step/sec: 9.21514
INFO:tensorflow:loss = 0.017158644, step = 404900 (10.851 sec)
I0910 06:43:04.261220 139802634913600 basic_session_run_hooks.py:260] loss = 0.017158644, step = 404900 (10.851 sec)
INFO:tensorflow:global_step/sec: 9.24815
I0910 0

INFO:tensorflow:Running local_init_op.
I0910 06:44:14.607657 139802634913600 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0910 06:44:14.738347 139802634913600 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Performing evaluation on 10 images.
I0910 06:44:24.918864 139794357131008 coco_evaluation.py:282] Performing evaluation on 10 images.
creating index...
index created!
INFO:tensorflow:Loading and preparing annotation results...
I0910 06:44:25.277890 139794357131008 coco_tools.py:116] Loading and preparing annotation results...
INFO:tensorflow:DONE (t=0.01s)
I0910 06:44:25.283595 139794357131008 coco_tools.py:138] DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.28s).
Accumulating evaluation results...
DONE (t=0.03s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.745
 Average Precision  (AP) @[ IoU=0.50      | area=  

INFO:tensorflow:global_step/sec: 9.19422
I0910 06:46:59.592105 139802634913600 basic_session_run_hooks.py:692] global_step/sec: 9.19422
INFO:tensorflow:loss = 0.0026719775, step = 406900 (10.876 sec)
I0910 06:46:59.592880 139802634913600 basic_session_run_hooks.py:260] loss = 0.0026719775, step = 406900 (10.876 sec)
INFO:tensorflow:global_step/sec: 9.23136
I0910 06:47:10.424841 139802634913600 basic_session_run_hooks.py:692] global_step/sec: 9.23136
INFO:tensorflow:loss = 0.010698201, step = 407000 (10.833 sec)
I0910 06:47:10.425966 139802634913600 basic_session_run_hooks.py:260] loss = 0.010698201, step = 407000 (10.833 sec)
INFO:tensorflow:global_step/sec: 9.27236
I0910 06:47:21.209579 139802634913600 basic_session_run_hooks.py:692] global_step/sec: 9.27236
INFO:tensorflow:loss = 0.00087090867, step = 407100 (10.785 sec)
I0910 06:47:21.210724 139802634913600 basic_session_run_hooks.py:260] loss = 0.00087090867, step = 407100 (10.785 sec)
INFO:tensorflow:global_step/sec: 9.17932
I0910

In [ ]:
# aquí nos vamos a fijar en los archivos model.ckpt y vamos a buscar el que tenga el número más alto, ya que este número
# indica el step del proceso de entrenamiento en el que se realizó el checkpoint del modelo
!dir training_bb

In [ ]:
# por si no lo ves claro, corriendo esto te dirá qué debes poner en el script siguiente que generará el grafo de inferencia
import glob
import os
max_step=0
for file in glob.glob("training_bb/*"):
    if ("model.ckpt" in  file):
        step=int (file.split(os.sep)[1].split(".")[1].split("-")[1])
        max_step= step if step>max_step else max_step
print("En el campo trained_checkpoint debes poner lo siguiente: \ntraining_bb/model.ckpt-"+str(max_step))

In [ ]:
# ahora vamos a exportar este modelo como un grafo de inferencia. Este grafo se podrá usar para generar predicciones luego.
# Debemos sustituir los parámetros de entrada ["pipeline_config_path","trained_checkpoint_prefix","output_directory"]
# por los nuestros. Es recomendable elegir un nombre reconocible para la carpeta de salida, como por ejemplo
# "inference_graph" seguido del modelo entrenado y  la fecha y la hora
!python3 models/research/object_detection/export_inference_graph.py --input_type image_tensor \
--pipeline_config_path training_bb/faster_rcnn_inception_v2_coco.config \
--trained_checkpoint_prefix training_bb/model.ckpt-2744\
--output_directory inference_graph_faster_rcnn_02_09_10_14

In [ ]:
!dir inference_graph_02_09_10_14

# Pediciendo en imágenes

In [ ]:
# debemos sustituir el parámetro -m por la carpeta donde está nuestro modelo exportado 
!python3 scripts/predict_on_images.py \
-i images_bb/test \
-o predicions_image_bb \
-m inference_graph_02_09_10_14 \
-l training_bb/label_map.pbtxt